In [1]:
%run ./apiMethods.ipynb
%run ./arenaCrawlers.ipynb

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 4.05 µs
CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 4.77 µs
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs
CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 5.01 µs
CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 4.05 µs
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs
CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 6.2 µs


In [11]:
def createAllTMsJSONandId(allTms):
    allTmJsonsWithid = []
    for machine in allTms:
        index = ((int(machine)-1)*18) + 1
        allTmJsonsWithid.append((requestMachine(index), machine))
    return allTmJsonsWithid


In [12]:
def createAllMoveJSONfromTMJSON(allTmJson):
    allMovesByTmJsons = []
    for tmJson in allTmJson:
        allMovesByTmJsons.append( (requestMove(tmJson[0]['move']['name']), tmJson[1] ) )
    return allMovesByTmJsons

In [63]:
allTmJson = createAllTMsJSONandId(POSSIBLE_TM_ARENA9)
allMoveJson = createAllMoveJSONfromTMJSON(allTmJson)

In [85]:
def appendTmMovesToMovesOfInterestCSV(moveJsonsWithId, moveTable):
    movesToAppend = []
    movesOfInterestColumNames = ['name', 'moveType', 'damageClass', 'power', 'pp', 'priority', 'accuracy']
    
    for moveJsonWithId in moveJsonsWithId:
        moveJson = moveJsonWithId[0]
        name = moveJson['name']
        if name not in moveTable.name:
            # extracts moves data to CSV format
            moveObject = produceMoveWithoutFetch(moveJson)
            moveCSVformat = [moveObject['name'],moveObject['moveType'],moveObject['damageClass'],moveObject['power'],moveObject['pp'],moveObject['priority'],moveObject['accuracy']]
            moveTable = moveTable.append({'name':moveObject['name'], 'moveType':moveObject['moveType'],'damageClass': moveObject['damageClass'],'power': moveObject['power'],'pp': moveObject['pp'], 'priority':moveObject['priority'],'accuracy':moveObject['accuracy']},ignore_index=True)
    return moveTable
            # ----- #
    #TODO WRITE table ToFile 
    #index=False

In [86]:
movesOfInterestTable = pd.read_csv(f'Data/red/arena-{1}/MovesOfInterest.csv')
movesOfInterestTable.drop(columns=movesOfInterestTable.columns[0], axis=1, inplace=True)
movesOfInterestTable = appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
movesOfInterestTable

,name,moveType,damageClass,power,pp,priority,accuracy
0,gust,flying,special,40.0,35,0,100.0
1,sand-attack,ground,status,NaN,15,0,100.0
2,quick-attack,normal,physical,40.0,30,1,100.0
3,whirlwind,normal,status,NaN,20,-6,NaN
4,wing-attack,flying,physical,60.0,35,0,100.0
...,...,...,...,...,...,...,...
104,metronome,normal,status,None,10,0,None
105,fire-blast,fire,special,110,5,0,85
106,fissure,ground,physical,None,5,0,30
107,sky-attack,flying,physical,140,5,0,90


In [47]:
movesOfInterestTable.loc[movesOfInterestTable['name']=='bubble-beam']

,Unnamed: 0,name,moveType,damageClass,power,pp,priority,accuracy


In [91]:
#allTmJson = createAllTMsJSONandId(POSSIBLE_TM_ARENA9)
#allMoveJson = createAllMoveJSONfromTMJSON(allTmJson)

# remove -1 wenn arena 9 gecrawled
for i in range(0, len(machineArrays)-1 ):
    arrayIndex = i
    arenaIndex = i+1
    print('ARENA ' , arenaIndex)
    pathMoves = f'Data/red/arena-{arenaIndex}/MovesOfInterest.csv'
    pathPokemon = f'Data/red/arena-{arenaIndex}/EncounterablePokemon.csv'
    movesOfInterestTable = pd.read_csv(pathMoves)
    movesOfInterestTable.drop(columns=movesOfInterestTable.columns[0], axis=1, inplace=True)
    movesOfInterestTable = appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
    appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
    print(movesOfInterestTable.tail())
    
    encounterablePokemonTable = pd.read_csv(pathPokemon)    
    
    pokeToNewMoves = {}
    tmIdToMoveAndAllowedPoke = createIdToMoveNameAndAllowedPokemon(allMoveJson, encounterablePokemonTable)
    for tmId in machineArrays[arrayIndex]:
        tmIdNumber = tmId
        allowedPoke = tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']
        moveName = tmIdToMoveAndAllowedPoke[tmIdNumber]['name']
        for pokeName in tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']:
            if pokeName in pokeToNewMoves.keys():
                pokeToNewMoves[pokeName].append(moveName)
            else:
                 pokeToNewMoves[pokeName] = [moveName]
    newMoveArrays = []
    for name in encounterablePokemonTable.name:
        if name in pokeToNewMoves.keys():
            newMoveArrays.append(pokeToNewMoves[name])
        else:
            newMoveArrays.append([])
    encounterablePokemonTable['movesByTM'] = newMoveArrays
    
    
    encounterablePokemonTable.to_csv(pathPokemon,index=False) 
    movesOfInterestTable.to_csv(pathMoves,index=False) 


ARENA  1
           name moveType damageClass power  pp  priority accuracy
104   metronome   normal      status  None  10         0     None
105  fire-blast     fire     special   110   5         0       85
106     fissure   ground    physical  None   5         0       30
107  sky-attack   flying    physical   140   5         0       90
108   explosion   normal    physical   250   5         0      100
ARENA  2
           name moveType damageClass power  pp  priority accuracy
139   metronome   normal      status  None  10         0     None
140  fire-blast     fire     special   110   5         0       85
141     fissure   ground    physical  None   5         0       30
142  sky-attack   flying    physical   140   5         0       90
143   explosion   normal    physical   250   5         0      100
ARENA  3
           name moveType damageClass power  pp  priority accuracy
145   metronome   normal      status  None  10         0     None
146  fire-blast     fire     special   110   5   

In [275]:
idToMoveAndPokemon = createIdToMoveNameAndAllowedPokemon(allMoveJson, 1)

In [88]:
def createIdToMoveNameAndAllowedPokemon(allMoveJsonAndId, encounterablePokemonTable):
    idObject = {}
    for moveJson in allMoveJsonAndId:
        tmId = moveJson[1]
        moveJson = moveJson[0]
        moveName = moveJson['name']
        allowedPoke = []
        for pokemon in moveJson['learned_by_pokemon']:
            name = pokemon['name']
            if(encounterablePokemonTable.name.str.contains(name).any()):
                allowedPoke.append(name)
        idObject[tmId] = {"name":moveName,'allowedPoke':allowedPoke} 
    return idObject
                 #  pokemonToTM[name].append(moveName)